In [ ]:
%pip install tqdm

ERROR: unknown command "isntall" - maybe you meant "install"



In [ ]:
import numpy as np
import random
from tqdm import tqdm

## Helper and metric functions

In [ ]:
def is_permutation_sbox(S):
    """Check if S is a valid permutation S-box of size 256."""
    if len(S) != 256:
        return False
    return set(S) == set(range(256))

def fwht(a):
    """In-place Fast Walsh–Hadamard Transform of array a."""
    a = a.copy()
    h = 1
    n = len(a)
    while h < n:
        for i in range(0, n, h * 2):
            for j in range(i, i + h):
                x = a[j]
                y = a[j + h]
                a[j] = x + y
                a[j + h] = x - y
        h *= 2
    return a

def component_function(S, v):
    """Compute the component function f_v(x) = parity(S[x] & v) for S-box S and vector v."""
    f = np.zeros(256, dtype=int)
    for x in range(256):
        f[x] = bin(S[x] & v).count("1") & 1
    return f

def boolean_nonlinearity(f):
    """Compute the nonlinearity of a Boolean function f: {0,1}^8 -> {0,1}."""
    vec = np.array([1 if bit == 0 else -1 for bit in f], dtype=int)
    spectrum = fwht(vec)
    M = np.max(np.abs(spectrum))
    return 128 - M // 2

def min_component_nonlinearity(S):
    """Compute the minimum nonlinearity among all component functions of S-box S."""
    min_nl = 128
    for v in range(1, 256):
        f = component_function(S, v)
        nl = boolean_nonlinearity(f)
        if nl < min_nl:
            min_nl = nl
            if min_nl <= 96:
                break
    return min_nl

def differential_uniformity(S):
    """Compute the differential uniformity of S-box S."""
    du = 0
    for dx in range(1, 256):
        counts = np.zeros(256, dtype=int)
        for x in range(256):
            dy = S[x] ^ S[x ^ dx]
            counts[dy] += 1
        local_max = counts.max()
        if local_max > du:
            du = local_max
    return du

def fixed_points(S):
    """Count the number of fixed points in S-box S."""
    return sum(1 for x in range(256) if S[x] == x)

def opposite_fixed_points(S):
    """Count the number of opposite fixed points in S-box S."""
    return sum(1 for x in range(256) if S[x] == (x ^ 0xFF))

def algebraic_degree_boolean(f):
    """Compute the algebraic degree of a Boolean function f: {0,1}^8 -> {0,1}."""
    a = f.copy()
    n = 8
    for i in range(n):
        step = 1 << i
        for j in range(256):
            if j & step:
                a[j] ^= a[j ^ step]
    deg = 0
    for i in range(256):
        if a[i]:
            deg = max(deg, bin(i).count("1"))
    return deg

def sbox_algebraic_degree(S):
    """Compute the maximum algebraic degree among the output bits of S-box S."""
    max_deg = 0
    for bit in range(8):
        f = np.array([(S[x] >> bit) & 1 for x in range(256)], dtype=int)
        deg = algebraic_degree_boolean(f)
        if deg > max_deg:
            max_deg = deg
    return max_deg

def sac_metric(S):
    """Compute the Strict Avalanche Criterion (SAC) metric for S-box S. (mean and max deviation from 0.5 and the probability matrix)"""
    flips = np.zeros((8, 8), dtype=int)
    trials = 256

    for x in range(256):
        y = S[x]
        for i in range(8):
            x2 = x ^ (1 << i)
            y2 = S[x2]
            diff = y ^ y2
            for j in range(8):
                flips[i, j] += (diff >> j) & 1

    probs = flips / trials
    deviation = np.abs(probs - 0.5)

    return {
        "mean_dev": deviation.mean(),
        "max_dev": deviation.max(),
        "matrix": probs
    }

def bic_metric(S):
    """Compute the Bit Independence Criterion (BIC) metric for S-box S. (mean and max absolute correlation between output bits when input bits are flipped)"""
    correlations = []

    for i in range(8):
        flips = np.zeros((8, 256), dtype=int)

        for x in range(256):
            y = S[x]
            y2 = S[x ^ (1 << i)]
            diff = y ^ y2
            for j in range(8):
                flips[j, x] = (diff >> j) & 1

        for j in range(8):
            for k in range(j + 1, 8):
                a = flips[j]
                b = flips[k]

                a_mean = a.mean()
                b_mean = b.mean()

                num = np.sum((a - a_mean) * (b - b_mean))
                den = np.sqrt(
                    np.sum((a - a_mean) ** 2) *
                    np.sum((b - b_mean) ** 2)
                )

                if den > 0:
                    correlations.append(abs(num / den))

    correlations = np.array(correlations)

    return {
        "mean_corr": correlations.mean(),
        "max_corr": correlations.max()
    }

def random_sbox():
    """Generate a random permutation S-box of size 256."""
    S = np.arange(256, dtype=int)
    np.random.shuffle(S)
    return S

def evaluate_sbox(S):
    """Evaluate S-box S and return its cryptographic properties."""
    if not is_permutation_sbox(S):
        raise ValueError("Input S is not a valid permutation S-box of size 256.")

    sac = sac_metric(S)
    bic = bic_metric(S)
    return {
        "min_nl": min_component_nonlinearity(S),
        "du": differential_uniformity(S),
        "degree": sbox_algebraic_degree(S),
        "fixed": fixed_points(S),
        "opposite_fixed": opposite_fixed_points(S),
        "sac_mean_dev": sac["mean_dev"],
        "sac_max_dev": sac["max_dev"],
        # "sac_matrix": sac["matrix"],
        "bic_mean_corr": bic["mean_corr"],
        "bic_max_corr": bic["max_corr"]
    }

## Optimization functions

In [38]:
def fitness_tuple(m):
    """Convert evaluation metrics dictionary m into a fitness tuple."""
    return (
        m["min_nl"],
        -m["du"],
        -(m["fixed"] + m["opposite_fixed"]),
        -m["sac_mean_dev"],
        -m["bic_mean_corr"]
    )

def mutate_swap(S):
    """Mutate S-box S by swapping two random elements."""
    S2 = S.copy()
    i, j = random.sample(range(256), 2)
    S2[i], S2[j] = S2[j], S2[i]
    return S2

def random_sbox_restarts(n_restarts=10):
    """Generate multiple random S-boxes and return the best one found."""
    best_S = None
    best_metrics = None

    for _ in range(n_restarts):
        S = random_sbox()
        metrics = evaluate_sbox(S)
        if best_S is None or fitness_tuple(metrics) > fitness_tuple(best_metrics):
            best_S = S
            best_metrics = metrics

    return best_S, best_metrics

def hill_climb(
    max_steps=1000,
    max_no_improve=100
):
    S = random_sbox()
    best_metrics = evaluate_sbox(S)
    best_fit = fitness_tuple(best_metrics)

    no_improve = 0

    history = []

    for step in range(max_steps):
        S_candidate = mutate_swap(S)
        metrics = evaluate_sbox(S_candidate)
        fit = fitness_tuple(metrics)

        if fit > best_fit:
            S = S_candidate
            best_fit = fit
            best_metrics = metrics
            no_improve = 0
        else:
            no_improve += 1

        history.append(best_metrics["min_nl"])

        if no_improve >= max_no_improve:
            break

    return S, best_metrics, history

def hill_climb_restarts(n_restarts=10):
    """Perform hill climbing with multiple random restarts."""
    best_overall = None
    best_metrics = None
    best_history = None

    for _ in range(n_restarts):
        S, metrics, history = hill_climb()
        print(f"metrics['min_nl'] = {metrics['min_nl']}")
        if best_overall is None or fitness_tuple(metrics) > fitness_tuple(best_metrics):
            best_overall = S
            best_history = history
            best_metrics = metrics


    return best_overall, best_metrics, best_history


## Evolutionary algorithm

In [89]:
def swap_mutation(S, strength=1):
    """Mutate S-box S with specified strength."""
    S2 = S.copy()

    if strength == 1:
        i, j = random.sample(range(256), 2)
        S2[i], S2[j] = S2[j], S2[i]

    elif strength == 2:
        i, j, k = random.sample(range(256), 3)
        S2[i], S2[j], S2[k] = S2[j], S2[k], S2[i]

    elif strength == 3:
        idx = random.sample(range(256), 5)
        vals = [S2[i] for i in idx]
        random.shuffle(vals)
        for i, v in zip(idx, vals):
            S2[i] = v

    return S2

def cycle_rotate_mutation(sbox, strength=1):
    """
    Rotate a random cycle of positions in the S-box.
    sbox: list or numpy array of 256 elements
    strength: length of the cycle to rotate
    """
    child = sbox.copy()
    # select unique positions for the cycle
    indices = random.sample(range(256), min(strength, 256))
    # rotate values by one position
    values = [child[i] for i in indices]
    rotated = values[-1:] + values[:-1]
    for idx, val in zip(indices, rotated):
        child[idx] = val
    return child

def segment_reverse_mutation(sbox, strength=2):
    """
    Reverse a contiguous segment of the S-box.
    
    sbox: list or numpy array of 256 elements
    strength: length of the segment to reverse (at least 2)
    """
    child = sbox.copy()
    if strength < 2:
        strength = 2
    # choose start index
    start = random.randint(0, 256 - strength)
    end = start + strength
    # reverse the segment
    child[start:end] = child[start:end][::-1]
    return child


def tournament_select(population, fitnesses, k=5):
    """Select an individual from the population using tournament selection."""
    idx = random.sample(range(len(population)), k)
    best = idx[0]
    for i in idx[1:]:
        if fitnesses[i] > fitnesses[best]:
            best = i
    return population[best]

def evolutionary_search(
    pop_size=50,
    generations=200,
    elite_size=3,
    mutation_strengths=(1, 2, 3, 4),
    stagnation_limit=20
):
    """Mutation-based evolutionary search with adaptive mutation strength for S-boxes."""
    
    population = [random_sbox() for _ in range(pop_size)]
    metrics = [evaluate_sbox(S) for S in population]
    fitnesses = [fitness_tuple(m) for m in metrics]

    best_idx = max(range(pop_size), key=lambda i: fitnesses[i])
    best_S = population[best_idx]
    best_fit = fitnesses[best_idx]

    history = []
    stagnation_counter = 0

    for gen in range(generations):
        new_population = []

        elite_indices = sorted(range(len(population)), key=lambda i: fitnesses[i], reverse=True)[:elite_size]
        new_population.extend(population[i] for i in elite_indices)

        if stagnation_counter >= stagnation_limit:
            current_strengths = tuple(strength + 1 for strength in mutation_strengths)
        else:
            current_strengths = mutation_strengths

        while len(new_population) < pop_size:
            parent = tournament_select(population, fitnesses)
            strength = random.choice(current_strengths)
            if stagnation_counter < stagnation_limit:
                child = swap_mutation(parent, strength)
            else:
                op = random.choice([swap_mutation, segment_reverse_mutation, cycle_rotate_mutation])
                child = op(parent, strength)

            # child = segment_reverse_mutation(parent, strength)
            
            new_population.append(child)

        population = new_population
        metrics = [evaluate_sbox(S) for S in population]
        fitnesses = [fitness_tuple(m) for m in metrics]

        gen_best_idx = max(range(len(fitnesses)), key=lambda i: fitnesses[i])
        gen_best_fit = fitnesses[gen_best_idx]

        if gen_best_fit[0] <= best_fit[0] and gen_best_fit[1] <= best_fit[1]:
            stagnation_counter += 1
        else:
            stagnation_counter = 0

        if gen_best_fit > best_fit:
            best_fit = gen_best_fit
            best_S = population[gen_best_idx]
        

        history.append(best_fit[0])

        if gen % 10 == 0:
            print(f"Gen {gen}: best NL = {best_fit[0]}, best DU = {-best_fit[1]}, stagnation={stagnation_counter}")

    return best_S, evaluate_sbox(best_S), history

In [90]:
# sanity check
S, metrics, history = evolutionary_search()

print("Best S-box metrics found:")
for k, v in metrics.items():
    print(f"{k}: {v}")
print("History of minimum nonlinearity during evolutionary search:")
print(history)

Gen 0: best NL = 96, best DU = 10, stagnation=1
Gen 10: best NL = 96, best DU = 10, stagnation=11
Gen 20: best NL = 96, best DU = 10, stagnation=21
Gen 30: best NL = 98, best DU = 10, stagnation=1
Gen 40: best NL = 98, best DU = 10, stagnation=11
Gen 50: best NL = 98, best DU = 10, stagnation=21
Gen 60: best NL = 98, best DU = 10, stagnation=31
Gen 70: best NL = 98, best DU = 10, stagnation=41
Gen 80: best NL = 98, best DU = 10, stagnation=51
Gen 90: best NL = 98, best DU = 10, stagnation=61
Gen 100: best NL = 100, best DU = 10, stagnation=2
Gen 110: best NL = 100, best DU = 10, stagnation=12
Gen 120: best NL = 100, best DU = 10, stagnation=22
Gen 130: best NL = 100, best DU = 10, stagnation=32
Gen 140: best NL = 100, best DU = 10, stagnation=42
Gen 150: best NL = 100, best DU = 10, stagnation=52
Gen 160: best NL = 100, best DU = 10, stagnation=62
Gen 170: best NL = 100, best DU = 10, stagnation=72
Gen 180: best NL = 100, best DU = 10, stagnation=82
Gen 190: best NL = 100, best DU = 10